In [2]:
import pandas as pd
from meteostat import Stations, Point, Monthly, Daily
from datetime import date, datetime
import time
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("../files/03-areas_country.csv", index_col=0)
df.head()

,latitude,longitude,town,region,country,price,water,wc,grey_water
0,39.06956,-0.40980,Carcaixent,Comunitat Valenciana,España,Free,yes,yes,yes
1,42.64159,-8.89696,Boiro,Galicia,España,Free,yes,yes,unknown
2,42.90715,0.35921,Arreau,Occitanie,France,Paying,yes,no,yes
3,38.39479,-0.41007,el Campello,Comunitat Valenciana,España,Paying,yes,yes,unknown
4,43.37179,-8.44443,A Coruña,Galicia,España,Free,yes,yes,unknown


### Average temperature and precipitation by month from 1-1-2000 70 31-12-2022

In [4]:
def weather_month(dataf, weat):
    
    # Dictionary to change the columns name
    month_dict = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
                7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"}

    data_list = [] # empty list to append the row dataframe

    for ind, row in dataf.iterrows(): # iterate over the rows of the dataf
        lat = row["latitude"] # we need the latitude and the longitude to get the nearby stations
        lon = row["longitude"]
    
        # set time period
        start = datetime(2000, 1, 1) 
        end = datetime(2022, 12, 31)

        # get nearby stations
        stations = Stations()
        stations = stations.nearby(lat,lon)

        # select a station with data for the selected period
        try:
            station = stations.fetch(4)
            station = station[station["daily_end"] > "2002-01-01"]
            station_name = station.index[0]
        except:
            station = stations.fetch(4)
            station_name = station.index[0]

        # get daily data
        data = Daily(station_name, start, end)
        data = data.fetch()
        data.reset_index(inplace=True)
        data["time"] = pd.to_datetime(data["time"]) # convert the "time" column to datetime
        data["month"] = data["time"].dt.month # extract the month from "time"

        t_media = round(data.groupby("month")[weat].mean(),2) # group by month and extract the average temperature
        df_temp = t_media.to_frame().rename_axis(None, axis=0).T # convert to dataframe
        
        # add columns for lat, long, town and country
        df_temp["latitude"] = lat 
        df_temp["longitude"] = lon
        df_temp["town"] = row["town"]
        df_temp["country"] = row["country"]
        data_list.append(df_temp) # append the dataframe to the list
    
    # concat all the dataframe y the list and rename columns
    dat = pd.concat(data_list, axis=0, ignore_index= True)
    dat.rename(columns=month_dict, inplace= True)

    return dat

### Weather data to obtain
- **"tavg"** for temperature (average air temperature in ºC)
- **"prcp"** for precipitation (daily precipitation total in mm)
- It's also posible to obtain *"snow"* (snow depth in mm), *"tmin"* (minimun air temperature in ºC), *"tmax"* (maximum air temperature in ºC), *"wdir"* (average wind direction in degrees), *"wspd"*, average wind speed in km/h)

In [59]:
# Create a dataf for the temperature
df_avgtemperature = weather_month(df, "tavg")

In [60]:
df_avgtemperature.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude,town,country
0,11.30,12.66,13.59,15.29,19.32,23.25,26.31,26.63,23.66,19.62,14.99,12.99,39.06956,-0.40980,Carcaixent,España
1,9.67,10.48,12.25,14.01,16.56,19.19,20.99,20.97,19.41,16.36,12.17,10.41,42.64159,-8.89696,Boiro,España
2,5.22,6.17,8.84,11.27,14.62,18.40,19.91,19.96,17.30,13.86,8.55,6.17,42.90715,0.35921,Arreau,France
3,11.41,12.96,14.20,16.21,20.07,23.99,26.89,26.89,24.03,19.88,15.27,12.87,38.39479,-0.41007,el Campello,España
4,10.81,10.98,12.04,13.02,15.09,17.34,18.92,19.42,18.53,16.41,13.12,11.68,43.37179,-8.44443,A Coruña,España


In [61]:

df_avgtemperature.isnull().sum()

Jan          0
Feb          0
Mar          0
Apr          0
May          0
Jun          0
Jul          0
Aug          0
Sep          0
Oct          0
Nov          0
Dec          0
latitude     0
longitude    0
town         0
country      0
dtype: int64

In [62]:
df_avgtemperature.shape

(913, 16)

In [66]:
towns = df_avgtemperature.town.unique().tolist()

In [68]:
len(towns)

887

In [63]:
df_avgtemperature.town.value_counts()

Encamp                  3
Thann                   2
Campan                  2
Roma                    2
Ljubljana               2
                       ..
Jumièges                1
Kalmar                  1
Karlskrona              1
Kaysersberg-Vignoble    1
Cúllar                  1
Name: town, Length: 887, dtype: int64

In [69]:
town_list = df_avgtemperature.town.value_counts()[df_avgtemperature.town.value_counts()>1].index.tolist()

In [70]:
len(town_list)

25

In [71]:
df_avgtemperature = df_avgtemperature.drop_duplicates(subset=["town"])

In [72]:
df_avgtemperature.head(2)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude,town,country
0,11.30,12.66,13.59,15.29,19.32,23.25,26.31,26.63,23.66,19.62,14.99,12.99,39.06956,-0.40980,Carcaixent,España
1,9.67,10.48,12.25,14.01,16.56,19.19,20.99,20.97,19.41,16.36,12.17,10.41,42.64159,-8.89696,Boiro,España


In [73]:
df_avgtemperature.town.value_counts()

Carcaixent               1
Villefranche-d'Allier    1
Vercelli                 1
Vernet-les-Bains         1
Saint-Géry-Vers          1
                        ..
Ichtershausen            1
A Illa de Arousa         1
Vera                     1
Langon                   1
Cúllar                   1
Name: town, Length: 887, dtype: int64

In [74]:
df_avgtemperature.shape

(887, 16)

In [101]:
df_avgtemperature = df_avgtemperature.reset_index()

In [102]:
df_avgtemperature.to_csv("../files/04-average-temperature.csv") # Save to csv

In [89]:
df_precipitation = weather_month(df, "prcp") # Create a dataf for precipitation

In [90]:
df_precipitation.head(3)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude,town,country
0,1.16,0.94,1.65,1.35,1.57,0.89,0.23,0.55,2.35,2.22,2.11,1.19,39.06956,-0.40980,Carcaixent,España
1,6.03,4.46,4.71,4.79,3.13,2.03,1.29,1.64,2.22,7.01,7.07,7.06,42.64159,-8.89696,Boiro,España
2,3.43,2.86,2.60,3.17,3.26,2.38,2.03,1.88,2.25,2.28,3.72,2.93,42.90715,0.35921,Arreau,France


In [91]:
df_precipitation.isnull().sum()

Jan          126
Feb          126
Mar          128
Apr          127
May           13
Jun           13
Jul           13
Aug           13
Sep           13
Oct           13
Nov           13
Dec           13
latitude       0
longitude      0
town           0
country        0
dtype: int64

In [92]:
null_list = df_precipitation[df_precipitation.loc[:,"Jan":"Dec"].isnull().all(axis=1)].index.tolist()

In [93]:
null_list

[9, 66, 75, 133, 174, 175, 344, 425, 531, 560, 620, 669, 726]

In [94]:
df_precipitation.drop(null_list, axis=0, inplace=True)

In [96]:
df_precipitation.tail(3)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude,town,country
910,1.16,1.28,1.53,2.17,0.99,0.02,0.06,0.02,0.34,1.75,2.7,2.33,37.58107,-6.75165,Valverde del Camino,España
911,1.35,1.41,0.89,2.26,2.50,3.16,2.14,1.44,2.26,3.85,4.3,3.18,44.89777,5.36963,Vassieux-en-Vercors,France
912,0.67,0.61,0.96,0.64,0.40,0.10,0.01,0.09,0.53,0.76,0.8,1.13,37.55425,-2.60865,Cúllar,España


In [98]:
df_precipitation.tail(3)

,index,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,latitude,longitude,town,country
897,910,1.16,1.28,1.53,2.17,0.99,0.02,0.06,0.02,0.34,1.75,2.7,2.33,37.58107,-6.75165,Valverde del Camino,España
898,911,1.35,1.41,0.89,2.26,2.50,3.16,2.14,1.44,2.26,3.85,4.3,3.18,44.89777,5.36963,Vassieux-en-Vercors,France
899,912,0.67,0.61,0.96,0.64,0.40,0.10,0.01,0.09,0.53,0.76,0.8,1.13,37.55425,-2.60865,Cúllar,España


In [99]:
df_precipitation.town.value_counts()

Encamp                  3
København               2
Laruns                  2
Loudenvielle            2
Cangas                  2
                       ..
Kalmar                  1
Karlskrona              1
Kaysersberg-Vignoble    1
Kehl                    1
Cúllar                  1
Name: town, Length: 874, dtype: int64

In [100]:
df_precipitation = df_precipitation.drop_duplicates(subset=["town"])

In [103]:
df_precipitation = df_precipitation.reset_index()

In [104]:
df_precipitation.to_csv("../files/05-precipitation.csv") # save to csv